In [7]:
import pandas as pd

import plotly.express as px

from main import GameResult

In [8]:
def _sanity_check(x, CAT, animal_cols):
    cat_finished = x[CAT] == GameResult.FINISHED.name
    any_animal_finished = any(x[col] == GameResult.FINISHED.name for col in animal_cols)
    all_animals_chased = all(x[col] == GameResult.CHASED.name for col in animal_cols)
    assert cat_finished != any_animal_finished
    if any_animal_finished or all_animals_chased:
        assert any_animal_finished != all_animals_chased


def get_winner(x):
    CAT = "C"
    animal_cols = ["B", "S", "M"]
    _sanity_check(x, CAT, animal_cols)
    result = None
    for col in [CAT] + animal_cols:
        if x[col] == GameResult.FINISHED.name:
            if result is None:
                result = col
            else:
                raise ValueError("W?")
    return result

In [9]:
df = pd.read_csv("results.csv")
display(df.describe())
display(df.groupby(by="strategy").describe())

,epoch,position,game_result,shortcut_position,game_id,timestamp
count,440288.000000,440288.000000,440288.000000,330216.000000,440288.000000,4.402880e+05
mean,19.655253,15.486949,-0.460830,21.000000,1481.460453,1.746456e+12
std,13.532996,12.856758,6.568604,8.164978,868.341787,9.200312e+03
min,-1.000000,0.000000,-2.000000,11.000000,0.000000,1.746456e+12
25%,8.000000,6.000000,-2.000000,11.000000,727.000000,1.746456e+12
50%,19.000000,9.000000,-1.000000,21.000000,1470.000000,1.746456e+12
75%,30.000000,23.000000,-1.000000,31.000000,2239.000000,1.746456e+12
max,65.000000,50.000000,50.000000,31.000000,2999.000000,1.746456e+12


epoch                                                    \
                      count       mean        std  min  25%   50%   75%   max   
strategy                                                                        
CLOSEST_RUN_AWAY   142804.0  19.248705  13.432203 -1.0  8.0  18.0  30.0  65.0   
ONLY_ONE_RUN_AWAY  143816.0  19.362630  13.445531 -1.0  8.0  18.0  30.0  62.0   
RANDOM_SINGLE      153668.0  20.306921  13.682838 -1.0  8.0  19.0  32.0  61.0   

                   position             ... game_id         timestamp  \
                      count       mean  ...     75%     max     count   
strategy                                ...                             
CLOSEST_RUN_AWAY   142804.0  15.286484  ...  1754.0  1999.0  142804.0   
ONLY_ONE_RUN_AWAY  143816.0  15.331528  ...  2750.0  2999.0  143816.0   
RANDOM_SINGLE      153668.0  15.818700  ...   758.0   999.0  153668.0   

                                                                          \
                           mean          std           min           25%   
strategy                                                                   
CLOSEST_RUN_AWAY   1.746456e+12  2851.332337  1.746456e+12  1.746456e+12   
ONLY_ONE_RUN_AWAY  1.746456e+12  2983.360893  1.746456e+12  1.746456e+12   
RANDOM_SINGLE      1.746456e+12  3374.818131  1.746456e+12  1.746456e+12   

                                                             
                            50%           75%           max  
strategy                                                     
CLOSEST_RUN_AWAY   1.746456e+12  1.746456e+12  1.746456e+12  
ONLY_ONE_RUN_AWAY  1.746456e+12  1.746456e+12  1.746456e+12  
RANDOM_SINGLE      1.746456e+12  1.746456e+12  1.746456e+12  

[3 rows x 48 columns]

In [10]:
ds = []
for g_id in df.game_id.unique():
    df_game = df[df["game_id"] == g_id]
    last_epoch = df_game.epoch.max()
    df_last_epoch = df_game[df_game["epoch"] == last_epoch]
    d = {"end_epoch": last_epoch, "game_id": g_id, "strategy": df_last_epoch["strategy"].iloc[0]}
    for idx, row in df_last_epoch.iterrows():
        d[row["name"]] = GameResult(row["game_result"]).name
    ds.append(d)

df_game_result = pd.DataFrame(ds)
df_game_result["winner"] = df_game_result.apply(get_winner, axis=1)

In [11]:
df_game_result[df_game_result["C"] != GameResult.FINISHED.name]
px.histogram(df_game_result, "winner", color="strategy")


In [12]:
px.histogram(df_game_result, x="end_epoch", nbins=len(df_game_result.end_epoch.unique()), color="strategy")